In [8]:
import pandas as pd
from sklearn.metrics import *
from tqdm import tqdm
from utils import Load_Rumours_Dataset_filtering_since_first_post_Transfer_Learning
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier

In [5]:
train_dataset = 'charlie_hebdo'
test_dataset = 'germanwings_crash'
time_cut =10
processor = Load_Rumours_Dataset_filtering_since_first_post_Transfer_Learning(train_dataset,\
           test_dataset, time_cut=time_cut,test_size=0.7)

processor.load_data()
processor.process_data()
train,test = processor.get_final_dataframes()

rumour
0    10
1     3
Name: count, dtype: int64


In [3]:
def modelfit(alg, X_train, y_train, X_test, y_test, printFeatureImportance=True):
    with mlflow.start_run():
        # Fit the algorithm on the data
        #alg.fit(X_resampled, y_resampled)
            
        # Predict training set:
        dtrain_predictions = alg.predict(X_train)
        dtrain_predprob = alg.predict_proba(X_train)[:, 1]

        # Predict test set:
        dtest_predictions = alg.predict(X_test)
        dtest_predprob = alg.predict_proba(X_test)[:, 1]

        # Log model parameters
        mlflow.log_params(alg.get_params())

        # Log metrics for Train
        mlflow.log_metric("train_auc", roc_auc_score(y_train, dtrain_predprob))
        mlflow.log_metric("train_accuracy", accuracy_score(y_train, dtrain_predictions))
        mlflow.log_metric("train_precision", precision_score(y_train, dtrain_predictions))
        mlflow.log_metric("train_recall", recall_score(y_train, dtrain_predictions))

        # Log metrics for Test
        mlflow.log_metric("test_auc", roc_auc_score(y_test, dtest_predprob))
        mlflow.log_metric("test_accuracy", accuracy_score(y_test, dtest_predictions))
        mlflow.log_metric("test_precision", precision_score(y_test, dtest_predictions))
        mlflow.log_metric("test_recall", recall_score(y_test, dtest_predictions))

        # Print model report for Train
        print("\nModel Report Train")
        print("AUC Score : %f" % roc_auc_score(y_train, dtrain_predprob))
        print("Accuracy : %.4g" % accuracy_score(y_train, dtrain_predictions))
        print("Precision : %.4g" % precision_score(y_train, dtrain_predictions))
        print("Recall : %f" % recall_score(y_train, dtrain_predictions))

      

        # Print model report for Test
        print("\nModel Report Test")
        print("AUC Score : %f" % roc_auc_score(y_test, dtest_predprob))
        print("Accuracy : %.4g" % accuracy_score(y_test, dtest_predictions))
        print("Precision : %.4g" % precision_score(y_test, dtest_predictions))
        print("Recall : %f" % recall_score(y_test, dtest_predictions))


        # Log the model with input example
        #input_example = X_train[0:1]   # Use the first row of X_train as an example
        #mlflow.sklearn.log_model(alg, "model", input_example=input_example)
        mlflow.log_metric("time_cut", time_cut)
        mlflow.log_metric("test_size", test_size)
    
    # Explicitly end the MLflow run
    mlflow.end_run()

    return alg



In [11]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
#mlflow.set_experiment("spyder-experiment")
import mlflow.pytorch
mlflow.set_experiment("Random Forest SMOTE Filter Node 2024-12-29 Transfer Learning Ottawa shooting")

2025/02/22 15:15:20 INFO mlflow.tracking.fluent: Experiment with name 'Random Forest SMOTE Filter Node 2024-12-29 Transfer Learning Ottawa shooting' does not exist. Creating a new experiment.


<Experiment: artifact_location='/workspaces/rumour-detection-gnn/mlruns/31', creation_time=1740237320519, experiment_id='31', last_update_time=1740237320519, lifecycle_stage='active', name='Random Forest SMOTE Filter Node 2024-12-29 Transfer Learning Ottawa shooting', tags={}>

In [12]:
train_dataset = 'charlie_hebdo'
test_dataset = 'ottawashooting'
test_size=0.7

for time_cut in range(10,24*3*60,15):
    print(time_cut)
    
    processor = Load_Rumours_Dataset_filtering_since_first_post_Transfer_Learning(train_dataset,\
           test_dataset, time_cut=time_cut,test_size=test_size)
    
    processor.load_data()
    processor.process_data()
    train,test = processor.get_final_dataframes()

    X_train  = train.drop(columns=['rumour'])
    X_train = np.hstack([X_train.drop(columns=['embeddings_avg']).values, np.array(pd.DataFrame(X_train.embeddings_avg.tolist()))])
    #X = np.hstack([X.drop(columns=['embeddings_avg']).values, np.array(pd.DataFrame(X.embeddings_avg.tolist()))])
    y_train =train['rumour']
    
    X_test  = test.drop(columns=['rumour'])
    X_test = np.hstack([X_test.drop(columns=['embeddings_avg']).values, np.array(pd.DataFrame(X_test.embeddings_avg.tolist()))])
    #X = np.hstack([X.drop(columns=['embeddings_avg']).values, np.array(pd.DataFrame(X.embeddings_avg.tolist()))])
    y_test =test['rumour']
    
    smote = SMOTE(random_state=42,sampling_strategy='minority')
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

    final_model =  RandomForestClassifier(
    n_estimators=100,      # Number of trees in the forest
    max_depth=5
    )

        
    final_model.fit(X_resampled, y_resampled)  
        
    modelfit(final_model,  X_resampled, y_resampled,X_test,y_test)

10
rumour
1    17
0    14
Name: count, dtype: int64

Model Report Train
AUC Score : 0.963117
Accuracy : 0.8908
Precision : 0.8551
Recall : 0.941035

Model Report Test
AUC Score : 0.941176
Accuracy : 0.871
Precision : 0.8421
Recall : 0.941176
25
rumour
1    40
0    29
Name: count, dtype: int64

Model Report Train
AUC Score : 0.961875
Accuracy : 0.892
Precision : 0.859
Recall : 0.938026

Model Report Test
AUC Score : 0.898276
Accuracy : 0.8986
Precision : 0.8667
Recall : 0.975000
40
rumour
0    62
1    57
Name: count, dtype: int64

Model Report Train
AUC Score : 0.963593
Accuracy : 0.8917
Precision : 0.8565
Recall : 0.941035

Model Report Test
AUC Score : 0.889926
Accuracy : 0.8403
Precision : 0.7969
Recall : 0.894737
55
rumour
0    80
1    72
Name: count, dtype: int64

Model Report Train
AUC Score : 0.963190
Accuracy : 0.895
Precision : 0.8613
Recall : 0.941637

Model Report Test
AUC Score : 0.878472
Accuracy : 0.8224
Precision : 0.7586
Recall : 0.916667
70
rumour
0    95
1    81
Name: 